# Test Hot Pixel Groups

This notebook tests an algorithm that finds hot pixels, and organizes them into hot pixel groups.

## Create the Test Data

In [1]:
# THIS COMMENT IS THE LONGEST A LINE CAN BE AND STILL RENDER COMPLETELY WHEN PRINTING IN LANDSCAPE MODE.

import numpy as np
import matplotlib.pyplot as plt
from math import floor
%matplotlib inline

from collections import namedtuple

HotPixel = namedtuple('HotPixel', 'x y value')
HotPixelGroup = namedtuple('HotPixelGroup', 'leader_index hot_pixels')

TEST_DATA_WIDTH = 6
TEST_DATA_HEIGHT = 4

TEST_DATA = np.zeros((TEST_DATA_HEIGHT, TEST_DATA_WIDTH))

TEST_HOT_PIXEL_GROUPS = [
    HotPixelGroup(1, [
        HotPixel(0, 0, 275), 
        HotPixel(0, 1, 300), # Group leader
        HotPixel(1, 1, 250)
    ]),
    HotPixelGroup(1, [
        HotPixel(3, 0, 275),
        HotPixel(4, 1, 275), # Group leader that tests tie-breaking algorithm
        HotPixel(3, 2, 225),
        HotPixel(4, 2, 250),
        HotPixel(5, 2, 225),
        HotPixel(5, 3, 325)
    ]),
    HotPixelGroup(0, [
        HotPixel(1, 3, 250)  # Isolated hot pixel
    ]),
    HotPixelGroup(5, [
        HotPixel(3, 0, 275), # Note that this group of six pixels is the same as one of the above
        HotPixel(4, 1, 275),
        HotPixel(3, 2, 225),
        HotPixel(4, 2, 250),
        HotPixel(5, 2, 225),
        HotPixel(5, 3, 325)  # Also a group leader
    ])   
]

for hot_pixel_group in TEST_HOT_PIXEL_GROUPS:
    hot_pixels = hot_pixel_group.hot_pixels
    for hot_pixel in hot_pixels:
        TEST_DATA[hot_pixel.y, hot_pixel.x] = hot_pixel.value


## Find Hot Pixel Leaders

In [2]:
def is_winner(candidate_leader, i, j, data):
    if candidate_leader.value > data[j, i]:
        return True
    elif candidate_leader[2] == data[j, i]:
        # some nasty tie-breaking
        if candidate_leader[0] > i:
            return True
        elif candidate_leader[0] == i and candidate_leader[1] > j:
            return True
    else:
        return False

def is_leader(candidate_leader, data):
    data_height, data_width = data.shape
    for offset_y in [-1, 0, 1]:
        j = floor(candidate_leader.y + offset_y)
        if j < 0 or j >= data_height:
            continue
        for offset_x in [-1, 0, 1]:
            i = floor(candidate_leader.x + offset_x)
            # we don't compare the candidate wih itself
            if offset_x == 0 and offset_y == 0:
                continue
            if i < 0 or i >= data_width:
                continue
            if not is_winner(candidate_leader, i, j, data):
                return False
    return True

def find_hot_pixel_leaders(data, threshold=200):
    # first we simply find all the hot pixels
    data_height, data_width = data.shape
    exceedances = data > threshold  # an array of true-false values
    values_of_exceedances = data[exceedances]
    exceedance_indices = np.nonzero(exceedances)  # a crafty way of getting the indices of the exceedances
    # all of the hot pixels are candidate leaders
    candidate_leaders = np.transpose([exceedance_indices[1], exceedance_indices[0], values_of_exceedances])
    leaders = []
    for i in range(candidate_leaders.shape[0]):
        row = candidate_leaders[i]
        candidate_leader = HotPixel(row[0], row[1], row[2])
        if is_leader(candidate_leader, data):
            leaders.append(candidate_leader)
    return leaders

find_hot_pixel_leaders(TEST_DATA, 200)

[HotPixel(x=0.0, y=1.0, value=300.0),
 HotPixel(x=4.0, y=1.0, value=275.0),
 HotPixel(x=1.0, y=3.0, value=250.0),
 HotPixel(x=5.0, y=3.0, value=325.0)]

In [ ]:
# Display the representative subtracted dark

def display_leader(data, leader):
    fig, axes = plt.subplots(1, 1, figsize=(8, 8))

    lower_x = floor(leader.x - 5)
    upper_x = floor(lower_x + 11)
    slice_x = slice(lower_x, upper_x)
    lower_y = floor(leader.y - 5)
    upper_y = floor(lower_y + 11)
    slice_y = slice(lower_y, upper_y)
    
    title = "x={}:{}, y={}:{} of Subtracted Dark".format(lower_x, upper_x, lower_y, upper_y)
    
    subframe = representative_dark_data[lower_y:upper_y, lower_x:upper_x]
    
    axes.imshow(subframe, cmap='gray')
    axes.set_title(title)
    plt.tight_layout()
    plt.show()

for leader in hot_pixel_leaders:
    if leader.x >= 400 and leader.x < 500 and leader.y >= 1700 and leader.y < 1800:
        display_leader(representative_dark_data, leader)
